# TPP Attachment Limit investigation - Oct 2020

Hypothesis:
- TPP have increased the attachment limit and this has reduced the number of errors related to large attachments.

Context:
- TPP had a 60mb GP2GP attachment limit that should have been increased to 100mb over the last few months. We wanted to see if that change has reduced the number of errors related to large messages. 
- To note, we are unable to access the MI Data past Feb 2020, but we are able to access the SPINE logs. Therefore the following is based on data analysis from the SPINE logs. But there were also issues with the SPINE logs since migration to Spine Cloud, which are noted below.


Assumptions:
- YGA is TPP

Requirements:
- This notebook requires febuary spine logs to be stored as "feb.csv.gz" in the data folder

In [29]:
from typing import List
from datetime import datetime
from dateutil.tz import tzutc
from dateutil.relativedelta import relativedelta

from gp2gp.spine.models import Conversation
from gp2gp.date.range import DateTimeRange
from gp2gp.io.json import write_json_file, read_json_file, upload_json_object
from gp2gp.odsportal.sources import construct_practice_list_from_dict
from gp2gp.pipeline.dashboard.main import read_spine_csv_gz_files
from gp2gp.spine.transformers import group_into_conversations

In [30]:
def isYGA(code):
    print(f"code: {code}")
    return code == "YGA"

In [31]:
def calculate_large_attachment_errors(conversations):
    count = 0
    for conversation in conversations:
        conversation_errors = []
        large_error = False
        for message in conversation.messages:
            if message.error_code in [23, 25, 29, 30, 31]:
                large_error = True
                conversation_errors.append(message.error_code)
        if large_error is True:
            count +=1
        if len(conversation_errors) > 1:
            print("Conversation ", conversation.id ," has multiple errors - ", conversation_errors)
    return count

In [32]:
metric_month = datetime(2020, 2, 1, tzinfo=tzutc())
next_month = metric_month + relativedelta(months=1)
time_range = DateTimeRange(metric_month, next_month)

In [33]:
practice_data = read_json_file("../data/practice-list.json")
practice_metadata = construct_practice_list_from_dict(practice_data)

In [34]:
spine_messages = read_spine_csv_gz_files(["../data/feb.csv.gz"])

In [35]:
conversations = group_into_conversations(spine_messages)

In [36]:
conversation_list = list(conversations)

print("-- All conversations --")
print("Conversations: ", len(conversation_list))
large_attachment_errors = calculate_large_attachment_errors(conversation_list)
print("Large attachment failures: ", large_attachment_errors)

-- All conversations --
Conversations:  220001
Conversation  96A80060-5A1F-11EA-8779-31BB08329F86  has multiple errors -  [29, 31]
Conversation  E03B3730-5A1D-11EA-87BA-E9F10B35C116  has multiple errors -  [29, 31]
Conversation  0586B9F0-5A15-11EA-9A17-0F09BBBC8FAD  has multiple errors -  [29, 31]
Conversation  381614C0-5884-11EA-9D8E-A34EE459F0D9  has multiple errors -  [29, 31, 29, 31, 29, 31]
Conversation  EB411BCB-FE90-4E8C-8B8F-64678B0B413D  has multiple errors -  [29, 31]
Conversation  BFB56300-599B-11EA-A6E5-0BDF5EA7C7BF  has multiple errors -  [29, 31]
Conversation  05F383C0-598C-11EA-A4ED-B338B00DC0B7  has multiple errors -  [29, 31]
Conversation  EE09E940-5989-11EA-9C89-E7F39F2CD23A  has multiple errors -  [29, 31]
Conversation  C5B5F890-597E-11EA-BE96-E569A80F5FF4  has multiple errors -  [29, 31]
Conversation  F30ADE10-4E74-11EA-9765-C30FB92F0EF7  has multiple errors -  [30, 29, 31]
Conversation  57B6D1F0-597B-11EA-9969-4F39AD4B9491  has multiple errors -  [29, 31]
Conversat

In [37]:
print("\n-- Conversations involving TPP --")

without_yga_list = []
for conversation in conversation_list:
    if (
        conversation.messages[0].from_party_ods_code == "YGA"
        or conversation.messages[0].to_party_ods_code == "YGA"
    ):
        without_yga_list.append(conversation)

print("Conversations: ", len(without_yga_list))
large_attachment_errors = calculate_large_attachment_errors(without_yga_list)
print("Large attachment errors: ", large_attachment_errors)


-- Conversations involving TPP --
Conversations:  77991
Conversation  96A80060-5A1F-11EA-8779-31BB08329F86  has multiple errors -  [29, 31]
Conversation  E03B3730-5A1D-11EA-87BA-E9F10B35C116  has multiple errors -  [29, 31]
Conversation  0586B9F0-5A15-11EA-9A17-0F09BBBC8FAD  has multiple errors -  [29, 31]
Conversation  381614C0-5884-11EA-9D8E-A34EE459F0D9  has multiple errors -  [29, 31, 29, 31, 29, 31]
Conversation  BFB56300-599B-11EA-A6E5-0BDF5EA7C7BF  has multiple errors -  [29, 31]
Conversation  05F383C0-598C-11EA-A4ED-B338B00DC0B7  has multiple errors -  [29, 31]
Conversation  EE09E940-5989-11EA-9C89-E7F39F2CD23A  has multiple errors -  [29, 31]
Conversation  C5B5F890-597E-11EA-BE96-E569A80F5FF4  has multiple errors -  [29, 31]
Conversation  F30ADE10-4E74-11EA-9765-C30FB92F0EF7  has multiple errors -  [30, 29, 31]
Conversation  57B6D1F0-597B-11EA-9969-4F39AD4B9491  has multiple errors -  [29, 31]
Conversation  A20D5AF0-3C3C-11EA-8E18-75940952EF55  has multiple errors -  [30, 29,